In [1]:
# 從arXiv導入搜索功能
from arxiv import Client, Search, SortCriterion
import requests 
from bs4 import BeautifulSoup

In [18]:
# 創建默認的 API 客戶端
client = Client()

# 定義 arXiv 搜索條件
arxiv_search = Search(
    query="RAG",
    max_results=20,
    sort_by=SortCriterion.SubmittedDate
)
results = client.results(arxiv_search)
results

In [21]:
# 遍歷搜索結果並打印相關信息
for result in results:
    paper_id = result.get_short_id()  # 短id
    paper_title = result.title  # 文章標題
    paper_url = result.entry_id  # 鏈接
    paper_summary = result.summary.replace("\n", "")  # 文章摘要去掉換行符
    paper_first_author = result.authors[0]  # 文章的第一作者
    publish_time = result.published.date()  # 發表的發布日期
    update_time = result.updated.date()  # 文章的更新時間
    paper_stars = get_stars(get_paper_code_url(paper_id))
    #print(str(paper_stars) + "\n")

In [17]:
paper_zh_summary = translate_zh_to_en(paper_summary)
paper_zh_summary

c:\Users\HCWang\.conda\envs\arxiv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-edge.cognitive.microsofttranslator.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'大规模语言模型（LLM）在各种语言任务中取得了显著的成功，但存在幻觉和时间错位。为了缓解这些缺点，检索增强生成（RAG）已被用于提供外部知识以促进答案生成。然而，由于缺乏特定领域的知识和现实世界场景的复杂性，将此类模型应用于医疗领域面临着一些挑战。在这项研究中，我们探索了具有 RAG 框架的 LLM，用于医学领域的知识密集型任务。为了评估 LLM 的能力，我们引入了 MedicineQA，这是一个多轮对话基准，模拟真实世界的药物咨询场景，并要求 LLM 回答从医学数据库中检索到的证据。MedicineQA 包含 300 个多轮问答对，每个问答对都嵌入在详细的对话历史中，突出了这项知识密集型任务对当前 LLM 构成的挑战。我们进一步提出了一个新的 \\textit{Distill-Retrieve-Read} 框架，而不是之前的 \\textit{Retrieve-then-Read}。具体来说，提炼和检索过程利用工具调用机制来制定搜索查询，以模拟搜索引擎使用的基于关键字的查询。通过实验结果，我们表明，我们的框架带来了显着的性能改进，并在证据检索过程中超越了以前的同行，在证据检索准确性方面。这一进步揭示了将RAG应用于医疗领域。'

In [6]:
def get_paper_code_url(paper_id):
    """
    獲取論文的 GitHub 代碼 URL

    Args:
        paper_id (str): 論文 ID

    Returns:
        str: GitHub 代碼 URL 或 None

    Example:
        >>> get_paper_code_url("12345")
        "https://github.com/paperswithcode/paperswithcode/blob/master/code/12345.py"
    """

    base_url = "https://arxiv.paperswithcode.com/api/v0/papers/"
    code_url = base_url + paper_id

    code_response = requests.get(code_url, verify=False).json()

    if "official" in code_response and code_response["official"]:
        github_code_url = code_response["official"]["url"]
        return github_code_url

    return None

In [7]:
def get_stars(github_code_url):
    """
    獲取 GitHub 的星星數

    Args:
        github_code_url (str): GitHub URL

    Returns:
        int: 星星數

    Example:
        >>> get_stars("https://github.com/paperswithcode/")
        1234
    """
    try:
        # 訪問 GitHub 上的代碼倉庫主頁
        code_html = requests.get(github_code_url, verify=False)

        # 利用 BeautifulSoup 庫解析主頁，爬取標星數
        soup = BeautifulSoup(code_html.text, "html.parser")

        # 標星數的 HTML 元件
        a_stars = soup.find_all("a", href=github_code_url.split("https://github.com")[-1] + "stargazers")

        # 獲取 GitHub 上的項目標星數
        stars = a_stars[0].text.strip().split("\n")[0]

    except Exception:
        stars = 0

    return int(stars)

In [8]:
def get_edge_token():
    auth_url = "https://edge.microsoft.com/translate/auth"
    auth_headers = {
        "Content-Type": "text/plain; charset=utf-8"
    }
    Authorization = requests.get(auth_url, headers=auth_headers).text
    return Authorization

In [9]:
def url_option():
    url = "https://api-edge.cognitive.microsofttranslator.com/translate?from=en&to=zh-CHS&api-version=3.0&includeSentenceLength=true"
    #headers = { 用户的headers }
    # 发送请求
    # requests.options(url, headers=headers)
    requests.options(url)

In [10]:
#调用翻译API
import json
def get_translate(token, body):
    # 翻译API
    translate_url = "https://api-edge.cognitive.microsofttranslator.com/translate?from=en&to=zh-CHS&api-version=3.0&includeSentenceLength=true"
    translate_headers = {
        "Authorization": "Bearer " + str(token),
        "Content-Length": str(len(json.dumps(body, separators=(",", ":")))),
        "Content-Type": "application/json"
    }
    res = requests.post(translate_url, headers=translate_headers, data=json.dumps(body, separators=(",", ":")), verify=False)
    return res

In [11]:
def translate_zh_to_en(contents):
    trans_contents = []
    trans_contents.append({"Text": contents})
    token = get_edge_token()  # 获取翻译API的使用Token
    url_option()  # 向API发送预检信息
    # 翻译摘要内容
    trans_text = get_translate(token, body=trans_contents).json()
    return trans_text[0]["translations"][0]["text"]